In [28]:
import numpy as np
import pandas as pd
import spacy
nlp = spacy.load("ru_core_news_sm")
import tensorflow as tf
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc, mean_squared_error
from sklearn.model_selection import train_test_split
from json import load


In [29]:
not_discriptors = list()
with open('../../Data/Words/word_pairs_not_disc.txt', 'r', encoding='utf-8') as txt:
    for i in txt:
        not_discriptors.append(i.replace('\n', ''))

with open('../../Data/Events/type_of_events.json', 'r', encoding='utf-8') as js:
    events = load(js)

with open('../../Data/Datasets/data.json', 'r', encoding='utf-8') as js:
    discriptors = load(js)


In [30]:
dict_of_disc = {'expession': [], 'is_discr': []}
for subject in discriptors.values():
    for d in sum(subject.values(), []):
        dict_of_disc['expession'].append(nlp(d).vector)
        dict_of_disc['is_discr'].append(True)

for subject in not_discriptors:
    dict_of_disc['expession'].append(nlp(subject).vector)
    dict_of_disc['is_discr'].append(False)



In [31]:
features = np.array(dict_of_disc['expession'])
labels = np.array(list(map(int, dict_of_disc['is_discr'])))


In [32]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=71)


In [33]:
import tensorflow as tf

model_class = tf.keras.models.Sequential()
model_class.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*1.5), activation='softmax',
                                      input_shape=(X_train[0].shape[0],), name='input'))
model_class.add(tf.keras.layers.Dense(
    int(X_train[0].shape[0]*2), activation='relu', name='hidden_layer_2'))
model_class.add(tf.keras.layers.Dense(
    int(X_train[0].shape[0]*3), activation='relu', name='hidden_layer_3'))
model_class.add(tf.keras.layers.Dense(
    int(X_train[0].shape[0]*2), activation='relu', name='hidden_layer_4'))
model_class.add(tf.keras.layers.Dense(
    int(X_train[0].shape[0]*1.5), activation='relu', name='hidden_layer_5'))
model_class.add(tf.keras.layers.Dense(
    int(X_train[0].shape[0]*0.5), activation='relu', name='hidden_layer_6'))
model_class.add(tf.keras.layers.Dense(1, name='output'))
model_class.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Dense)               (None, 144)               13968     
                                                                 
 hidden_layer_2 (Dense)      (None, 192)               27840     
                                                                 
 hidden_layer_3 (Dense)      (None, 288)               55584     
                                                                 
 hidden_layer_4 (Dense)      (None, 192)               55488     
                                                                 
 hidden_layer_5 (Dense)      (None, 144)               27792     
                                                                 
 hidden_layer_6 (Dense)      (None, 48)                6960      
                                                                 
 output (Dense)              (None, 1)                

In [34]:
model_class.compile(
    loss='mse',  # Функция потерь
    optimizer='Adam',  # Оптимизатор
    metrics=[  # Метрики
        'mse',  # Если у объекта назначено имя, то можно вызвать объект с его помощью
        # tf.keras.metrics.Precision()
    ]
)


In [35]:
model_class.fit(
    X_train,  # Набор входных данных
    y_train,  # Набор правильных ответов
    validation_split=0.4,  # Этот параметр автоматически выделит часть обучающего набора на валидационные данные. В данном случа 20%
    epochs=40,  # Процесс обучения завершится после 10 эпох
    # Набор данных будет разбит на пакеты (батчи) по 8 элементов набора в каждом.
    batch_size=2
)


Epoch 1/40
239/239 [==============================] - 2s 3ms/step - loss: 0.1661 - mse: 0.1661 - val_loss: 0.0712 - val_mse: 0.0712
Epoch 2/40
239/239 [==============================] - 1s 3ms/step - loss: 0.0758 - mse: 0.0758 - val_loss: 0.0702 - val_mse: 0.0702
Epoch 3/40
239/239 [==============================] - 1s 3ms/step - loss: 0.0491 - mse: 0.0491 - val_loss: 0.0846 - val_mse: 0.0846
Epoch 4/40
239/239 [==============================] - 1s 3ms/step - loss: 0.0336 - mse: 0.0336 - val_loss: 0.0498 - val_mse: 0.0498
Epoch 5/40
239/239 [==============================] - 1s 3ms/step - loss: 0.0186 - mse: 0.0186 - val_loss: 0.0487 - val_mse: 0.0487
Epoch 6/40
239/239 [==============================] - 1s 3ms/step - loss: 0.0183 - mse: 0.0183 - val_loss: 0.0436 - val_mse: 0.0436
Epoch 7/40
239/239 [==============================] - 1s 3ms/step - loss: 0.0088 - mse: 0.0088 - val_loss: 0.0415 - val_mse: 0.0415
Epoch 8/40
239/239 [==============================] - 1s 3ms/step - loss: 0.

In [36]:
predict = model_class.predict(X_test)


9/9 [==============================] - 0s 2ms/step


In [37]:
mean_squared_error(predict, y_test)


0.053351143183825

In [38]:
predictA = model_class.predict(features)
print('Accuracy')
print(accuracy_score(labels, predictA > 0.5))
print('Confusion matrix')
print(confusion_matrix(labels, predictA > 0.5))
print('Precision, Recall, F\n', classification_report(
    labels, predictA > 0.5))


34/34 [==============================] - 0s 1ms/step
Accuracy
0.9652582159624413
Confusion matrix
[[328  30]
 [  7 700]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.98      0.92      0.95       358
           1       0.96      0.99      0.97       707

    accuracy                           0.97      1065
   macro avg       0.97      0.95      0.96      1065
weighted avg       0.97      0.97      0.96      1065



In [55]:
print('Accuracy')
print(accuracy_score(y_test, predict > 0.98))
print('Confusion matrix')
print(confusion_matrix(y_test, predict > 0.98))
print('Precision, Recall, F\n', classification_report(
    y_test, predict > 0.98))


Accuracy
0.9400749063670412
Confusion matrix
[[ 89   7]
 [  9 162]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.91      0.93      0.92        96
           1       0.96      0.95      0.95       171

    accuracy                           0.94       267
   macro avg       0.93      0.94      0.94       267
weighted avg       0.94      0.94      0.94       267



In [61]:
tn, fp, fn, tp = confusion_matrix(y_test, predict > 0.9).ravel()
(tn, fp, fn, tp)


(86, 10, 3, 168)

In [48]:
model_class.predict(np.array([nlp('рекомендации, чтобы поддерживать внутреннюю').vector]))


1/1 [==============================] - 0s 20ms/step


array([[0.9850821]], dtype=float32)

In [49]:
model_class.predict(
    np.array([nlp('чтобы поддерживать внутреннюю').vector]))


1/1 [==============================] - 0s 18ms/step


array([[0.97055477]], dtype=float32)

In [51]:
model_class.predict(
    np.array([nlp('поддерживать внутреннюю структуру').vector]))


1/1 [==============================] - 0s 22ms/step


array([[0.9897646]], dtype=float32)

In [40]:
model_class.save('../../PipeLines/Classifications/checker_is_discriptor_spacy_vectorize')


INFO:tensorflow:Assets written to: ../../PipeLines/Classifications/checker_is_discriptor_spacy_vectorize\assets


INFO:tensorflow:Assets written to: ../../PipeLines/Classifications/checker_is_discriptor_spacy_vectorize\assets
